In [1]:
# Ensure deduped census CSV exists (create if missing)
import os
import pandas as pd

src_candidates = ['census_datacleaning/census_population_2022.csv', 'census_population_2022.csv']
src = next((p for p in src_candidates if os.path.exists(p)), None)
if src is None:
    raise FileNotFoundError('Source census CSV not found; expected one of: ' + ', '.join(src_candidates))

out_paths = ['census_population_2022_deduped.csv', 'census_datacleaning/census_population_2022_deduped.csv']
if any(os.path.exists(p) for p in out_paths):
    existing = [p for p in out_paths if os.path.exists(p)][0]
    print('Deduped census CSV already exists at:', existing)
else:
    print('Reading source:', src)
    df = pd.read_csv(src)

    # map lowercase column names to original names
    col_map = {c.lower(): c for c in df.columns}

    # heuristics to find district and town columns
    def find_col(key_parts):
        for orig_lower, orig in col_map.items():
            if all(k in orig_lower for k in key_parts):
                return orig
        return None

    district_col = find_col(['census', 'district']) or find_col(['district'])
    town_col = find_col(['city', 'town']) or find_col(['village']) or find_col(['citytownvillage'.lower()])

    if district_col is None or town_col is None:
        # fallback: try common exact names
        for cand in ['Census District', 'census_district', 'District']:
            if cand in df.columns:
                district_col = cand
                break
        for cand in ['City/Town/Village', 'city/town/village', 'city_town_village', 'City/Town']:
            if cand in df.columns:
                town_col = cand
                break

    if district_col is None or town_col is None:
        print('Warning: could not confidently detect district/town columns. Using first two columns as keys.')
        district_col, town_col = df.columns[0], df.columns[1]

    print('Using key columns:', district_col, 'and', town_col)
    df[district_col] = df[district_col].astype(str).str.strip()
    df[town_col] = df[town_col].astype(str).str.strip()

    df_dedup = df.drop_duplicates(subset=[district_col, town_col], keep='first').copy()
    print('Rows before:', len(df), 'after dedup:', len(df_dedup))

    for p in out_paths:
        try:
            df_dedup.to_csv(p, index=False)
            print('Wrote deduped census file to', p)
        except Exception as e:
            print('Failed to write', p, e)


Deduped census CSV already exists at: census_population_2022_deduped.csv


In [9]:
# Reproducible removal of known-bad rows from deduped census CSV
# Correction: the provided numbers were file line numbers (as shown in VSCode) where line 1 is the header.
# Map file-line L -> dataframe row index = L - 2 (because pandas row 0 == file line 2).
import os
import shutil
import pandas as pd
from datetime import datetime

candidates = ['census_population_2022_deduped.csv', 'census_datacleaning/census_population_2022_deduped.csv']
path = next((p for p in candidates if os.path.exists(p)), None)
if path is None:
    raise FileNotFoundError('Deduped census CSV not found; expected one of: ' + ', '.join(candidates))

# Find original backup created earlier (non-timestamped .bak or the most recent timestamped bak)
bak_candidates = [path + '.bak']
# also look for timestamped bak files
bak_dir = os.path.dirname(path) or '.'
for name in os.listdir(bak_dir):
    if name.startswith(os.path.basename(path) + '.bak.'):
        bak_candidates.append(os.path.join(bak_dir, name))

bak_exists = [b for b in bak_candidates if os.path.exists(b)]
if bak_exists:
    # pick the most recent bak (timestamped or not)
    bak = sorted(bak_exists)[-1]
    print('Restoring from backup:', bak)
    shutil.copy(bak, path)
    print('Restored', path, 'from', bak)
else:
    print('No backup found among candidates:', bak_candidates)
    print('Proceeding with current file (be careful)')

# rows to remove provided as file-line numbers (1-based, header=1)
file_line_numbers = [2,136,195,209,239,248,261,267,294,343,361,367,401,427,449,494,535,553,570,614,658,682,716,726,748,750,773,788]

# load df
print('Loading', path)
df = pd.read_csv(path)
print('Rows before:', len(df))

# convert file-line numbers to df zero-based indices: df_idx = L - 2
df_indices = [L - 2 for L in file_line_numbers]
# keep only valid indices
valid_df_indices = [i for i in df_indices if 0 <= i < len(df)]
invalid_file_lines = [file_line_numbers[i] for i, idx in enumerate(df_indices) if idx < 0 or idx >= len(df)]
if invalid_file_lines:
    print('Warning: some requested file-line positions are out of range and will be skipped:', invalid_file_lines)

# map to actual index labels to drop
to_drop_idx = [df.index[i] for i in valid_df_indices]
print('Dropping file-line positions (1-based):', [file_line_numbers[i] for i, idx in enumerate(df_indices) if 0 <= idx < len(df)])

# create a new timestamped backup of the file we're about to overwrite
ts = datetime.now().strftime('%Y%m%dT%H%M%S')
new_backup = f"{path}.pre_correctdrop.{ts}.bak"
shutil.copy(path, new_backup)
print('Created pre-correction backup:', new_backup)

# drop and reset index
df2 = df.drop(index=to_drop_idx).reset_index(drop=True)
print('Rows after drop:', len(df2))

# write cleaned file
try:
    df2.to_csv(path, index=False)
    print('Wrote corrected cleaned deduped file to', path)
except Exception as e:
    print('Failed to write', path, e)

# show a small sample around the first removed file-line (if valid)
if valid_df_indices:
    first_df_i = valid_df_indices[0]
    # after removal, the neighborhood index will shift — show a neighborhood near where the first removal was.
    start = max(0, first_df_i - 3)
    end = min(len(df2), start + 8)
    print('\nSample rows (after corrected removal):')
    display(df2.iloc[start:end])
else:
    print('No valid positions to remove.')


Restoring from backup: census_population_2022_deduped.csv.bak
Restored census_population_2022_deduped.csv from census_population_2022_deduped.csv.bak
Loading census_population_2022_deduped.csv
Rows before: 8319
Dropping file-line positions (1-based): [2, 136, 195, 209, 239, 248, 261, 267, 294, 343, 361, 367, 401, 427, 449, 494, 535, 553, 570, 614, 658, 682, 716, 726, 748, 750, 773, 788]
Created pre-correction backup: census_population_2022_deduped.csv.pre_correctdrop.20251208T134449.bak
Rows after drop: 8291
Wrote corrected cleaned deduped file to census_population_2022_deduped.csv

Sample rows (after corrected removal):


,Census District,City/Town/Village,Total Population,Year
0,Gaborone,Kgale Hill Township Extension 1,"2,075",2022
1,Gaborone,Gaborone West Extension 2 (Phase 1),"2,017",2022
2,Gaborone,Gaborone West Extension 3 (Phase 1),"5,131",2022
3,Gaborone,Gaborone West Extension 4 (Phase 1),"4,917",2022
4,Gaborone,Gaborone West Extension 5 (Phase 1),"2,200",2022
5,Gaborone,Gaborone West Extension 6 (Phase 1),"5,088",2022
6,Gaborone,Gaborone West Extension 7 (Phase 1),"2,181",2022
7,Gaborone,Gaborone West Extension 8 (Phase 2),"1,641",2022


In [10]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

census = pd.read_csv("census_population_2022_deduped.csv")

# clean columns
census.columns = census.columns.str.strip().str.lower().str.replace(" ", "_")

# drop duplicates to avoid repeated lookups
towns = census[["census_district", "city/town/village"]].drop_duplicates().copy()

# combine district and town for better search accuracy
towns["query"] = towns["city/town/village"] + ", " + towns["census_district"] + ", Botswana"


In [11]:
# THIS CODE TAKES >4 HOURS TO RUN DUE TO RATE LIMITING DO NOT RUN UNLESS YOU ARE ABSOLUTELY SURE

geolocator = Nominatim(user_agent="botswana_census_geocoder")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)  # stay polite

tqdm.pandas()

# get coordinates
towns["location"] = towns["query"].progress_apply(geocode)
towns["latitude"] = towns["location"].apply(lambda loc: loc.latitude if loc else None)
towns["longitude"] = towns["location"].apply(lambda loc: loc.longitude if loc else None)


  0%|          | 21/8291 [00:30<4:19:19,  1.88s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('Gaborone West Extension 21 (Phase 4), Gaborone, Botswana',), **{}).
Traceback (most recent call last):
  File "/opt/anaconda3/envs/amr_opt/lib/python3.11/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/amr_opt/lib/python3.11/site-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/amr_opt/lib/python3.11/http/client.py", line 1395, in getresponse
    response.begin()
  File "/opt/anaconda3/envs/amr_opt/lib/python3.11/http/client.py", line 325, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/amr_opt/lib/python3.11/http/client.py", line 286, in _rea

In [12]:
towns.to_csv("census_villages_geocoded.csv", index=False)
# here, you will find that there are several missing coordinates due to limitations of Nominatim. 
# to improve coverage, we can use Google Maps Geocoding API as a secondary source. 
# see the geocode_google.py script in /botswana_geocode for that.

In [17]:
geo = pd.read_csv("../botswana_geocode/census_population_2022_geocoded_google_strict.csv")
pop = pd.read_csv("census_population_2022.csv")

# Create normalized merge key from both files
geo["merge_key"] = geo["City/Town/Village"].str.strip().str.lower()
pop["merge_key"] = pop["City/Town/Village"].str.strip().str.lower()

# Merge on the normalized key
merged = pop.merge(geo[["merge_key", "latitude", "longitude"]],
                   on="merge_key",
                   how="left")

# Drop the temporary merge key
merged = merged.drop(columns=["merge_key"])

merged.to_csv("census_population_with_coords.csv", index=False)
print(f"Merged {len(merged)} rows with coordinates")
print(f"Rows with coordinates: {merged['latitude'].notna().sum()}")


Merged 10484 rows with coordinates
Rows with coordinates: 10457


In [27]:
import pandas as pd
import pyreadstat

df_raw, meta = pyreadstat.read_sav("botswanacensusmicrodata.sav")
df_raw.shape

(2359609, 128)

In [29]:
df_raw["NEW_AGE"].describe()
(df_raw["NEW_AGE"] == 999).sum()


np.int64(2695)

In [31]:
df = df_raw.copy()
df.shape

before = df.shape[0]
df = df[df["NEW_AGE"] != 999]
after = df.shape[0]

before, after, before - after

(2359609, 2356914, 2695)

In [ ]:
df["DistrictCode"] = df["District"].astype(int)
sorted(df["DistrictCode"].unique())

[np.int64(1),
 np.int64(2),
 np.int64(3),
 np.int64(4),
 np.int64(5),
 np.int64(6),
 np.int64(7),
 np.int64(10),
 np.int64(11),
 np.int64(12),
 np.int64(13),
 np.int64(20),
 np.int64(21),
 np.int64(30),
 np.int64(31),
 np.int64(32),
 np.int64(40),
 np.int64(50),
 np.int64(51),
 np.int64(52),
 np.int64(53),
 np.int64(54),
 np.int64(55),
 np.int64(56),
 np.int64(60),
 np.int64(70),
 np.int64(71),
 np.int64(72),
 np.int64(73),
 np.int64(74),
 np.int64(80),
 np.int64(81),
 np.int64(82),
 np.int64(90),
 np.int64(91)]

In [ ]:

district_map = {
    1: "Gaborone",
    2: "Francistown",
    3: "Lobatse",
    4: "Selebi-Phikwe",
    5: "Orapa",
    6: "Jwaneng",
    7: "Sowa Town",
    10: "Ngamiland East",
    11: "Barolong",
    12: "Ngwaketse West",
    13: "Ngwaketse Central",
    20: "South East",
    21: "Ngamiland West",
    30: "Kweneng East",
    31: "Kweneng West",
    32: "Kweneng Central",
    40: "Kgatleng",
    50: "Central Serowe-Palapye",
    51: "Central Mahalapye",
    52: "Central Bobonong",
    53: "Central Boteti",
    54: "Central Tutume",
    55: "Central Molalatau",
    56: "Central Madinare",
    60: "North East",
    70: "Kgalagadi South",
    71: "Kgalagadi North",
    72: "Ghanzi",
    73: "CKGR",
    74: "Ghanzi Farms",
    80: "Chobe",
    81: "Kasane",
    82: "Pandamatenga",
    90: "Kgalagadi Rural",
    91: "Tsabong"
}

df["DistrictName"] = df["DistrictCode"].map(district_map)

In [37]:
df["DistrictName"].isna().sum()


np.int64(0)

In [38]:
sorted(df["DistrictName"].unique())


['Barolong',
 'CKGR',
 'Central Bobonong',
 'Central Boteti',
 'Central Madinare',
 'Central Mahalapye',
 'Central Molalatau',
 'Central Serowe-Palapye',
 'Central Tutume',
 'Chobe',
 'Francistown',
 'Gaborone',
 'Ghanzi',
 'Ghanzi Farms',
 'Jwaneng',
 'Kasane',
 'Kgalagadi North',
 'Kgalagadi Rural',
 'Kgalagadi South',
 'Kgatleng',
 'Kweneng Central',
 'Kweneng East',
 'Kweneng West',
 'Lobatse',
 'Ngamiland East',
 'Ngamiland West',
 'Ngwaketse Central',
 'Ngwaketse West',
 'North East',
 'Orapa',
 'Pandamatenga',
 'Selebi-Phikwe',
 'South East',
 'Sowa Town',
 'Tsabong']

In [39]:
bins = [-1, 1, 6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 200]

labels = [
    "<1", "1–5", "6–10", "11–15", "16–20",
    "21–25", "26–30", "31–35", "36–40",
    "41–45", "46–50", "51–55", "56–60",
    "61–65", "66+"
]

df["AgeGroup"] = pd.cut(df["NEW_AGE"], bins=bins, labels=labels, right=False)
df["AgeGroup"].isna().sum()
sorted(df["AgeGroup"].unique())

['11–15',
 '16–20',
 '1–5',
 '21–25',
 '26–30',
 '31–35',
 '36–40',
 '41–45',
 '46–50',
 '51–55',
 '56–60',
 '61–65',
 '66+',
 '6–10',
 '<1']

In [41]:
# total population per district
district_totals = pop.groupby("DistrictName")["Population"].sum().reset_index()

district_totals[district_totals["Population"] == 0]
pop.groupby("DistrictName")["Population"].sum().sort_values()

DistrictName
Kasane                       486
CKGR                         949
Ghanzi Farms                1919
Sowa Town                   3264
Orapa                       8639
Pandamatenga               17506
Jwaneng                    18790
Tsabong                    23482
Ghanzi                     28707
Lobatse                    29778
Ngwaketse West             31967
Kgalagadi Rural            35322
Chobe                      38488
Selebi-Phikwe              42326
Kweneng West               52287
Ngwaketse Central          52311
South East                 55093
Ngamiland West             57113
Barolong                   59077
North East                 69299
Kgalagadi North            74830
Central Bobonong           77524
Central Boteti             78377
Ngamiland East             78627
Central Molalatau          80268
Central Tutume             85789
Central Serowe-Palapye     94252
Central Madinare          102882
Francistown               103285
Kgalagadi South           1204

In [43]:
df["DistrictCode"] = df["District"].astype(int)
df["DistrictName"] = df["DistrictCode"].map(district_map)

pop = (
    df.groupby(["DistrictCode","DistrictName","AgeGroup"])
      .size()
      .reset_index(name="Population")
)

pop = pop.sort_values(["DistrictCode","AgeGroup"])


/var/folders/0n/fdwzy91j2j38zqkrjnq3yc040000gn/T/ipykernel_4706/858037773.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["DistrictCode","DistrictName","AgeGroup"])


In [58]:
# rebuild district code and name cleanly
df["DistrictCode"] = df["District"].astype(int)
df["DistrictName"] = df["DistrictCode"].map(district_map)

# rebuild population table from scratch
pop = (
    df.groupby(["DistrictName","AgeGroup"])
      .size()
      .reset_index(name="Population")
)

# add district code column based on name
pop["DistrictCode"] = pop["DistrictName"].map(
    df[["DistrictName","DistrictCode"]].drop_duplicates().set_index("DistrictName")["DistrictCode"]
)

# sort
pop = pop.sort_values(["DistrictName","AgeGroup"]).reset_index(drop=True)

pop.to_csv("botswana_population_age_breakdown.csv", index=False)

/var/folders/0n/fdwzy91j2j38zqkrjnq3yc040000gn/T/ipykernel_4706/4210955656.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(["DistrictName","AgeGroup"])
